In [1]:
input_height = 12
input_width = 12
input_chan = 3
epochs = 1
learnrate =0.1
batchsize = 100
output_ckpt_path = './build/chkpts/float_model.ckpt'
infer_graph_path = './build/chkpts/inference_graph.pb'
tboard_path = './build/tb_logs'

In [2]:
import os
import sys
import argparse
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
from pathlib import Path
import pandas as pd
import glob
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import class_weight
import multiprocessing
import numpy as np
import json as json
from tensorflow.python.tools import freeze_graph
from tensorflow.keras.utils import plot_model, to_categorical

#import keras
#config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
#sess = tf.Session(config=config) 
#keras.backend.set_session(sess)

# Silence TensorFlow messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# workaround for TF1.15 bug "Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


class cfg:
    weights = {
        "Agriculture":                                               67256 ,
        "Airports": 0,
        "Annual crops associated with permanent crops" : 0,
        "Bare rock": 0,
        "Beaches, dunes, sands":0,
        "Burnt areas":0,
        "Coastal lagoons":0,
        "Construction sites": 0,
        "Continuous urban fabric": 0,
        "Discontinuous urban fabric":0,
        "Estuaries":0,
        "Fruit trees and berry plantations":0,
        "Green urban areas":0,
        "Land principally occupied by agriculture, with significant areas of natural vegetation":0,
        "Mineral extraction sites":0,
        "Inland marshes":0,
        "Agro-forestry areas":					  15790 ,
        "Arable land":						 100394 ,
        "Beaches,dunes,sands":					   1193 ,
        "Broad-leaved forest":					  73407 ,
        "Coastal wetlands":						   1033 ,
        "Complex cultivation patterns":				  53530 ,
        "Coniferous forest":					  86569 ,
        "Dump sites":0,
        "Industrial or commercial units":				   6182 ,
        "Inland waters":						  35349 ,
        "Inland wetlands":						  11620 ,
        "Intertidal flats":0,
        "Marine waters":						  39110 ,
        "Mixed forest":						  91926 ,
        "Moors and heathland":0,
        "Natural grassland":0,
        "Moors,heathlandandsclerophyllousvegetation":		   8434 ,
        "Naturalgrasslandandsparselyvegetatedareas":		   6663 ,
        "Non-irrigated arable land":0,
        "Olive groves":0,
        "Peatbogs":0,
        "Pastures":						  50977 ,
        "Permanent crops":				  15862 ,
        "Port areas":0,
        "Rice fields":0,
        "Road and rail networks and associated land":0,
        "Permanently irrigated land":0,
        "Salt marshes":0,
        "Salines":0,
        "Transitional woodland/shrub":				  77589 ,
        "Sclerophyllous vegetation":0,
        "Sea and ocean":0,
        "Urbanfabric":					  38779,
        "Sport and leisure facilities":0,
        "Sparsely vegetated areas":0,
        "Vineyards":0,
        "Water bodies":0,
        "Water courses":0
}




# Set up directories and files
INFER_GRAPH_DIR = os.path.dirname(infer_graph_path)
INFER_GRAPH_FILENAME =os.path.basename(infer_graph_path)

def get_data(path_images):
    classes_arr = []
    files = glob.glob(str(path_images) + "/*/" + '*.tif')

    for filename in files:
        out = Path(filename)
        parent = out.resolve().parent
        label_file = glob.glob(str(parent) + '/*.json')[0]

        with open(label_file) as json_file:
            data = json.load(json_file)
        name = data['labels']
        classes_arr.append([str(out), name])

    return pd.DataFrame(classes_arr, columns=['files', 'labels'])

# load data paths to dictionaries.
#root_path = "/root/tensorflow_datasets/"
root_path = "/workspace/data/"
dict_train = get_data(root_path+"train/")
dict_test = get_data(root_path+"test/")
dict_val = get_data(root_path+"val/")

mlb = MultiLabelBinarizer()
mlb.fit(dict_train['labels'])
print("Labels:")
for (i, label) in enumerate(mlb.classes_):
    print("{}. {}".format(i, label))


# Create Data Iterators
#Data generator for training data
train_datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = train_datagen.flow_from_dataframe(
    dict_train,
    x_col='files',
    y_col='labels',
    target_size=(input_height,input_width),
    batch_size=batchsize,
    class_mode='categorical',
)
### Now compute class weights from this

import sys
weight_sum = sum([cfg.weights[x] for x in cfg.weights])
print("Total Weight: %s" % weight_sum)
weights = {y:(cfg.weights[x] / weight_sum) for y,x in enumerate(mlb.classes_)}

#Data generator for test data
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_dataframe(
    dict_test,
    x_col='files',
    y_col='labels',
    target_size=(input_height,input_width),
    batch_size=batchsize,
    class_mode='categorical',
    shuffle=False
)

#Data generator for validation data
validation_datagen = ImageDataGenerator(rescale=1. / 255)
validation_generator = validation_datagen.flow_from_dataframe(
    dict_val,
    x_col='files',
    y_col='labels',
    target_size=(input_height,input_width),
    batch_size=batchsize,
    class_mode='categorical',
    shuffle=False
)

# get number of cores
num_cpus = multiprocessing.cpu_count()
print("Number of cores: %s" %num_cpus)


# Create a callback for storing the model. This will be used for quantization later
fname = os.path.sep.join([INFER_GRAPH_DIR, "best-model.ckpt"])
checkpoint = tf.keras.callbacks.ModelCheckpoint(fname,
                            monitor="val_loss", mode="min",
                            #monitor="val_acc", mode="max",
                            save_best_only=True, verbose=1)

#hub_layer = hub.KerasLayer("https://tfhub.dev/google/remote_sensing/bigearthnet-resnet50/1", trainable=False, input_shape=(12,12,3))#model = tf.keras.Sequential()
#model = tf.keras.Sequential()
#model.add(hub_layer)
#model.add(tf.keras.layers.Dense(43, activation='sigmoid'))

model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu",input_shape=(12,12,3)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100, activation='sigmoid'))
model.add(tf.keras.layers.Dense(43, activation='softmax'))

# Compile new model
model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.SGD(learning_rate=learnrate, momentum=0.9),
        metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()] 
)

# Training phase with training data
print("\n----------------------------",flush=True)
print(" TRAINING STARTED...",flush=True)
print("----------------------------",flush=True)

history = model.fit(
        train_generator,
        epochs=epochs,
        validation_steps=validation_generator.samples/validation_generator.batch_size,
        steps_per_epoch=train_generator.samples/train_generator.batch_size,
        validation_data=validation_generator,
        callbacks=[checkpoint], #checkpoint = ModelCheckpoint(fname
        workers=num_cpus,
        #class_weight=weights,
        verbose=1
)

Labels:
0. Agro-forestry areas
1. Airports
2. Annual crops associated with permanent crops
3. Bare rock
4. Beaches, dunes, sands
5. Broad-leaved forest
6. Burnt areas
7. Coastal lagoons
8. Complex cultivation patterns
9. Coniferous forest
10. Construction sites
11. Continuous urban fabric
12. Discontinuous urban fabric
13. Dump sites
14. Estuaries
15. Fruit trees and berry plantations
16. Green urban areas
17. Industrial or commercial units
18. Inland marshes
19. Intertidal flats
20. Land principally occupied by agriculture, with significant areas of natural vegetation
21. Mineral extraction sites
22. Mixed forest
23. Moors and heathland
24. Natural grassland
25. Non-irrigated arable land
26. Olive groves
27. Pastures
28. Peatbogs
29. Permanently irrigated land
30. Port areas
31. Rice fields
32. Road and rail networks and associated land
33. Salines
34. Salt marshes
35. Sclerophyllous vegetation
36. Sea and ocean
37. Sparsely vegetated areas
38. Sport and leisure facilities
39. Transit

In [3]:
#tf.keras.backend.set_learning_phase(0)

# fetch the tensorflow session using the Keras backend
#tf_session = tf.keras.backend.get_session()
# write out tensorflow checkpoint & meta graph
#saver = tf.compat.v1.train.Saver()
#save_path = saver.save(tf_session, output_ckpt_path)
#print (' Checkpoint created :', output_ckpt_path)

# set up tensorflow saver object
#saver = tf.compat.v1.train.Saver()
#sess = tf.compat.v1.keras.backend.get_session()
#saver.save(sess, output_ckpt_path)
#model.save(filepath= output_ckpt_path)
#tf.keras.models.save_model(model, filepath=INFER_GRAPH_DIR, save_format='tf')
#with tf.compat.v1.Graph().as_default():
    # define placeholders for the input data
    #x_1 = tf.compat.v1.placeholder(tf.float32, shape=[None,input_height,input_width,input_chan], name='input_1')

    # call the CNN function with is_training=False
    #customcnn = tf.keras.Model(inputs=model.inputs,outputs=model.get_layer(name="dense").output)
    #logits_1 = customcnn(cnn_in=x_1, is_training=False)

#    tf.io.write_graph(tf.compat.v1.get_default_graph().as_graph_def(), INFER_GRAPH_DIR, INFER_GRAPH_FILENAME, as_text=False)
#    print(' Saved binary inference graph to %s' % infer_graph_path)

#sess = tf.compat.v1.keras.backend.get_session()
#tf.compat.v1.train.Saver().save(sess=sess, save_path=INFER_GRAPH_DIR)
#tf.compat.v1.train.write_graph(graph_or_graph_def=sess.graph_def, logdir=INFER_GRAPH_DIR, name=infer_graph_path, as_text=False)
# Method 1
#freeze_graph.freeze_graph(input_graph=pbtxt_filepath, input_saver='', input_binary=False, input_checkpoint=ckpt_filepath, output_node_names='cnn/output', restore_op_name='save/restore_all', filename_tensor_name='save/Const:0', output_graph=pb_filepath, clear_devices=True, initializer_nodes='')

'''
from sklearn import metrics 
predictions = model.predict(test_generator, verbose=1)
ground_truth = test_generator.classes

y_true = MultiLabelBinarizer().fit_transform(ground_truth)
y_pred = np.rint(predictions)
y_score = predictions


f = metrics.f1_score(y_true, y_pred, average='samples')
p = metrics.precision_score(y_true, y_pred, average='samples')
r = metrics.recall_score(y_true, y_pred, average='samples')
AP = metrics.average_precision_score(y_true, y_score)
lrap = metrics.label_ranking_average_precision_score(y_true, y_score)
ji = metrics.jaccard_score(y_true, y_pred, average="samples")
#mAP = mAP(y_true, y_score)
hl = metrics.hamming_loss(y_true, y_pred)
rl = metrics.label_ranking_loss(y_true, y_score)
cov = metrics.coverage_error(y_true, y_score)

print("------------------------------")
print("F1: ", f)
print("presision: ",p)
print("Recall: ", r)
print("AP: ", AP) # edit to mAP
print("lrap: ", lrap)
print("Jaccard Index: ", ji)
#print("mAP", mAP)
print("Hamming Loss: ", hl)
print("Ranking Loss: ", rl)
print("Coverage Error: ", cov)
'''
'''
# Check the input and output name
print ("\n TF input node name:")
print(model.inputs)
print ("\n TF output node name:")
print(model.outputs)

fname = os.path.sep.join([INFER_GRAPH_DIR, "final-model.ckpt"])
tf.keras.experimental.export_saved_model(model, fname)

freeze_graph.freeze_graph(None,
                      None,
                      None,
                      None,
                      model.outputs[0].op.name,
                      None,
                      None,
                      os.path.join(INFER_GRAPH_DIR, "frozen_model.pb"),
                      False,
                      "",
                      input_saved_model_dir='build/chkpts/final-model.ckpt/')
'''

'\n# Check the input and output name\nprint ("\n TF input node name:")\nprint(model.inputs)\nprint ("\n TF output node name:")\nprint(model.outputs)\n\nfname = os.path.sep.join([INFER_GRAPH_DIR, "final-model.ckpt"])\ntf.keras.experimental.export_saved_model(model, fname)\n\nfreeze_graph.freeze_graph(None,\n                      None,\n                      None,\n                      None,\n                      model.outputs[0].op.name,\n                      None,\n                      None,\n                      os.path.join(INFER_GRAPH_DIR, "frozen_model.pb"),\n                      False,\n                      "",\n                      input_saved_model_dir=\'build/chkpts/final-model.ckpt/\')\n'

In [4]:
# save post-training checkpoint, this saves all the parameters of the trained network
print("\n----------------------------",flush=True)
print(" SAVING CHECKPOINT & GRAPH...",flush=True)
print("----------------------------",flush=True)

#model.build(input_shape=(None, 12,12,3))

#from keras.utils.vis_utils import plot_model
#print the CNN structure
from tensorflow.keras.utils import plot_model, to_categorical
model.summary()
plot_model(model , show_shapes=True, show_layer_names=True, expand_nested=True)

print (model.inputs)
print (model.outputs)


----------------------------
 SAVING CHECKPOINT & GRAPH...
----------------------------
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 10, 10, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               160100    
_________________________________________________________________
dense_1 (Dense)              (None, 43)                4343      
Total params: 166,235
Trainable params: 166,235
Non-trainable params: 0
_________________________________________________________________
[<tf.Tensor 'conv2d_input:0

In [5]:
fname = os.path.sep.join([INFER_GRAPH_DIR, "final_model.ckpt"])
tf.keras.experimental.export_saved_model(model, fname)

In [6]:
freeze_graph.freeze_graph(None,
                      None,
                      None,
                      None,
                      model.outputs[0].op.name,
                      None,
                      None,
                      os.path.join(INFER_GRAPH_DIR, "frozen_model.pb"),
                      False,
                      "",
                      input_saved_model_dir='build/chkpts/final_model.ckpt')

node {
  name: "conv2d_input"
  op: "Placeholder"
  attr {
    key: "_output_shapes"
    value {
      list {
        shape {
          dim {
            size: -1
          }
          dim {
            size: 12
          }
          dim {
            size: 12
          }
          dim {
            size: 3
          }
        }
      }
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
        dim {
          size: 12
        }
        dim {
          size: 12
        }
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "conv2d/kernel"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 3
          }
          dim {
            size: 3
          }
          dim {
            s

In [8]:
!vai_q_tensorflow quantize --input_frozen_graph ./build/chkpts/frozen_model.pb --output_dir ./build/quantize/ --input_nodes 'conv2d_input' --output_nodes 'dense_1/Softmax' --input_shapes "?,12,12,3" --input_fn image_input_fn.calib_input

INFO: Checking Float Graph...
INFO: Float Graph Check Done.
INFO: Calibrating for 100 iterations...
100% (100 of 100) |######################| Elapsed Time: 0:00:01 Time:  0:00:01
INFO: Calibration Done.
INFO: Generating Deploy Model...
INFO: Deploy Model Generated.
********************* Quantization Summary *********************      
INFO: Output:       
  quantize_eval_model: ./build/quantize/quantize_eval_model.pb       
  deploy_model: ./build/quantize/deploy_model.pb
